In [ ]:
import gc
import codecs
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from random import choice
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

from keras_bert import load_trained_model_from_checkpoint, Tokenizer

from keras.utils import to_categorical
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam

In [ ]:
train = pd.read_csv('raw_data/train.csv', sep='\t')
test = pd.read_csv('raw_data/test.csv', sep='\t')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.target.value_counts()

In [ ]:
train.stance.value_counts(dropna=False)

In [ ]:
train['text'] = train.text.fillna('')
test['text'] = test.text.fillna('')

In [ ]:
train['stance'] = train['stance'].map({'AGAINST':0, 'FAVOR':1, 'NONE':2})
train.head()

In [ ]:
maxlen = 256

config_path = '/data/zhengheng/nlp_pretrain_models/chinese-roberta-wwm-ext-l12-h768-a12/bert_config.json'
checkpoint_path = '/data/zhengheng/nlp_pretrain_models/chinese-roberta-wwm-ext-l12-h768-a12/bert_model.ckpt'
dict_path = '/data/zhengheng/nlp_pretrain_models/chinese-roberta-wwm-ext-l12-h768-a12/vocab.txt'

token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

class data_generator:
    def __init__(self, data, batch_size=8, shuffle=True):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            
            if self.shuffle:
                np.random.shuffle(idxs)
            
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y[:, 0, :]
                    [X1, X2, Y] = [], [], []
                    
def build_bert():
    bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

    for l in bert_model.layers:
        l.trainable = True

    x1_in = Input(shape=(None,))
    x2_in = Input(shape=(None,))

    x = bert_model([x1_in, x2_in])
    x = Lambda(lambda x: x[:, 0])(x)
    p = Dense(3, activation='softmax')(x)

    model = Model([x1_in, x2_in], p)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=Adam(1e-5),
                  metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
DATA_LIST = []
for data_row in train.iloc[:].itertuples():
    DATA_LIST.append((data_row.text, to_categorical(data_row.stance, 3)))
DATA_LIST = np.array(DATA_LIST)

DATA_LIST_TEST = []
for data_row in test.iloc[:].itertuples():
    DATA_LIST_TEST.append((data_row.text, to_categorical(0, 3)))
DATA_LIST_TEST = np.array(DATA_LIST_TEST)

In [ ]:
def run_cv(nfold, data, data_label, data_test):
    
    kf = KFold(n_splits=nfold, shuffle=True, random_state=1029).split(data)
    train_model_pred = np.zeros((len(data), 3))
    test_model_pred = np.zeros((len(data_test), 3))

    for i, (train_fold, test_fold) in enumerate(kf):
        X_train, X_valid, = data[train_fold, :], data[test_fold, :]
        
        model = build_bert()
        early_stopping = EarlyStopping(monitor='val_acc', patience=3)
        plateau = ReduceLROnPlateau(monitor="val_acc", verbose=1, mode='max', factor=0.5, patience=2)
        checkpoint = ModelCheckpoint('./' + str(i) + '.hdf5', monitor='val_acc', 
                                         verbose=2, save_best_only=True, mode='max', save_weights_only=True)
        
        train_D = data_generator(X_train, shuffle=True)
        valid_D = data_generator(X_valid, shuffle=True)
        test_D = data_generator(data_test, shuffle=False)
        
        model.fit_generator(
            train_D.__iter__(),
            steps_per_epoch=len(train_D),
            epochs=5,
            validation_data=valid_D.__iter__(),
            validation_steps=len(valid_D),
            callbacks=[early_stopping, plateau, checkpoint],
        )
        
        train_model_pred[test_fold, :] =  model.predict_generator(valid_D.__iter__(), steps=len(valid_D), verbose=1)
        test_model_pred += model.predict_generator(test_D.__iter__(), steps=len(test_D), verbose=1)
        
        del model; gc.collect()
        K.clear_session()
        
    return train_model_pred, test_model_pred

In [ ]:
train_model_pred, test_model_pred = run_cv(5, DATA_LIST, None, DATA_LIST_TEST)

In [ ]:
test_pred = [np.argmax(x) for x in test_model_pred]
test['y'] = test_pred

In [ ]:
submission = test[['y']]
submission['id'] = submission.index
submission['y'] = submission['y'].map({0:'AGAINST', 1:'FAVOR', 2:'NONE'})
submission = submission[['id', 'y']]
submission.head()

In [ ]:
submission.to_csv('submissions/submission_bert_ext_wwm_baseline.csv', index=False, header=False)

In [ ]:
!head 'submissions/submission_bert_ext_wwm_baseline.csv'